# End-to-End RAG System

**`Retrieval-Augmented Generation (RAG)` helps language models access up-to-date or domain-specific information by combining retrieval with generation. Instead of relying only on what the model was trained on, RAG first retrieves relevant documents from a knowledge base and then uses those to generate more accurate and grounded responses. This makes it especially useful for tasks where correctness and context matter.**

# Architecture

**In this notebook, we build a RAG pipeline based on PDF documents, focusing on the *Ontario Renters' Guide* and *Condominium Regulations*. By the end, the application will be able to answer questions about renting in Ontario while directly citing actual regulations in its responses.**

### PDF Loader
Loads and extracts text from one or more PDF documents.

### PDF Chunker
Splits the extracted text into smaller, manageable chunks for better indexing and retrieval.

### Embedder
Converts each text chunk into a numerical vector using a pre-trained embedding model.

### Vector Store
Stores the vectors in a searchable index for efficient similarity-based retrieval.

### Retrieval
Finds the most relevant chunks from the vector store based on a user's query.

### Generation
Uses a language model to generate a final answer by grounding it in the retrieved chunks.


# Notable Features

### Recursive loading of PDF files from disk
Automatically scans nested folders to load all PDF files without manual intervention.

### Database for tracking PDF files and efficient management
Maintains a record of PDFs with versioning and status tracking to avoid redundant processing.

### Three ways of chunking: free text, semantics, and hierarchy
Offers flexible chunking strategies including naive splitting, semantic similarity, and document structure-aware methods.

### Use of metadata in chunks to enhance context retrieval and generation
Associates chunks with source, section titles, and page numbers to improve answer grounding.

### Database for tracking chunks and metadata
Stores chunk data and associated metadata for fast lookup and update.

### Two ways of embedding: SentenceTransformer and OpenAI
Supports both local and API-based embedding generation depending on resources and preferences.

### Vector Store using Qdrant and two ways of context retrieval: KNN and MMR
Leverages Qdrant for storage and retrieval using either simple nearest neighbors or diversity-enhanced MMR.

### Efficient syncing between PDFs, chunks, and embeddings
Keeps track of changes and updates embeddings only when source documents or chunking strategy changes.

### Two ways of generation: OpenAI or vLLM deployed model
Allows switching between hosted OpenAI models and local inference via vLLM.

### Database for tracking chats with ability to resume old chats similar to ChatGPT
Stores conversation history with session IDs for persistent, resumable chats.

### Context window management by two ways of context shortening
clipping and summarization of older chat exchanges: Optimizes prompt size by either truncating or summarizing previous messages.


# Imports

In [1]:
import hashlib
import logging
from pathlib import Path
from typing import Dict, List, Optional, Union, Tuple, Literal, Any

from pydantic_settings import BaseSettings
from pypdf import PdfReader
from sqlmodel import Field, Session, SQLModel, create_engine, select, delete
import os
import json
from enum import Enum

from openai import OpenAI
from pydantic import BaseModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title
import time
from sqlalchemy import Column, JSON

from sentence_transformers import SentenceTransformer

from qdrant_client import QdrantClient
from qdrant_client.models import (
    VectorParams,
    Distance,
    PointStruct,
    Filter,
    FieldCondition,
    MatchText,
    MatchAny,
)
import torch
import numpy as np
import random

import requests
from datetime import datetime

import tiktoken
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage, BaseMessage
from transformers import pipeline as hf_pipeline

/home/electron/PycharmProjects/fine_tune_llm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Print Versions

In [2]:
import sys
import platform
import os
import subprocess


def print_env_info():
    print("\n🌱 Welcome to your Environment Info Report! 🌱\n")

    # Python & OS
    print(f"🐍 Python:       {platform.python_implementation()} {platform.python_version()}")
    print(f"💻 Platform:     {platform.system()} {platform.release()} ({platform.machine()})\n")

    # CUDA & GPU via PyTorch
    try:
        import torch
        cuda_avail = torch.cuda.is_available()
        print(f"🚀 CUDA Available: {cuda_avail}")
        if cuda_avail:
            print(f"   • CUDA Version:   {torch.version.cuda}")
            print(f"   • cuDNN Version:  {torch.backends.cudnn.version()}")
            n_gpus = torch.cuda.device_count()
            print(f"   • GPU Count:      {n_gpus}")
            for i in range(n_gpus):
                print(f"     - GPU {i}:      {torch.cuda.get_device_name(i)}")
    except ImportError:
        print("🚫 PyTorch not installed, skipping CUDA/GPU info")

    # nvcc (if available)
    try:
        out = subprocess.check_output(['nvcc', '--version'], stderr=subprocess.STDOUT)
        release = [l for l in out.decode().splitlines() if "release" in l]
        print(f"\n📦 nvcc: {release[-1].strip()}")
    except Exception:
        print("\n📦 nvcc: not found in PATH")

    # nvidia-smi
    try:
        out = subprocess.check_output([
            'nvidia-smi',
            '--query-gpu=name,driver_version,memory.total',
            '--format=csv,noheader'], stderr=subprocess.DEVNULL
        ).decode().strip().splitlines()
        print("\n📊 nvidia-smi info:")
        for line in out:
            print("   " + line)
    except Exception:
        print("\n📊 nvidia-smi: not available")

    # Helper to show versions
    def show_ver(label, import_name=None):
        try:
            m = __import__(import_name or label)
            v = getattr(m, '__version__', None) or getattr(m, 'VERSION', None) or str(m)
            print(f"🔖 {label:<15} version: {v}")
        except ImportError:
            print(f"🔖 {label:<15} not installed")

    # Popular ML/LLM libraries
    print("\n📚 Library versions:")
    libs = [
        ('torch',      None),
        ('torchvision', None),
        ('torchaudio',  None),
        ('transformers', None),
        ('accelerate',  None),
        ('trl',         'trl'),
        ('peft',        'peft'),
        ('deepspeed',   None),
        ('bitsandbytes', None),
        ('datasets',    'datasets'),
        ('evaluate',    'evaluate'),
        ('tokenizers',  None),
        ('sentencepiece', None),
        ('huggingface_hub', None),
        ('numpy',       'numpy'),
        ('scipy',       'scipy'),
        ('pandas',      'pandas'),
        ('scikit-learn','sklearn'),
        ('wandb',       'wandb'),
        ('tensorboard', 'tensorboard'),
        ('mlflow',      'mlflow'),
        ('sqlmodel',    'sqlmodel'),
    ]
    for label, name in libs:
        show_ver(label, name)

    # Conda env and key env vars
    print("\n🔧 Conda env:", os.getenv('CONDA_DEFAULT_ENV', '(none)'))
    important_vars = ['CUDA_HOME', 'CUDA_PATH', 'LD_LIBRARY_PATH', 'HF_HOME', 'HF_DATASETS_CACHE']
    print("\n🌐 Environment variables:")
    for var in important_vars:
        print(f"   - {var:<15} = {os.getenv(var, '')}")

    print("\n✨ All set! Keep growing and training with confidence! ✨\n")

print_env_info()


🌱 Welcome to your Environment Info Report! 🌱

🐍 Python:       CPython 3.12.7
💻 Platform:     Linux 6.11.0-26-generic (x86_64)

🚀 CUDA Available: True
   • CUDA Version:   12.4
   • cuDNN Version:  90100
   • GPU Count:      1
     - GPU 0:      NVIDIA GeForce RTX 4070 Laptop GPU

📦 nvcc: Cuda compilation tools, release 12.1, V12.1.105

📊 nvidia-smi info:
   NVIDIA GeForce RTX 4070 Laptop GPU, 570.133.07, 8188 MiB

📚 Library versions:
🔖 torch           version: 2.6.0+cu124
🔖 torchvision     version: 0.21.0+cu124
🔖 torchaudio      version: 2.6.0+cu124
🔖 transformers    version: 4.51.3
🔖 accelerate      version: 1.6.0
🔖 trl             version: 0.17.0
🔖 peft            version: 0.15.2
🔖 deepspeed       not installed
🔖 bitsandbytes    version: 0.45.5
🔖 datasets        version: 3.6.0
🔖 evaluate        version: 0.4.3
🔖 tokenizers      version: 0.21.1
🔖 sentencepiece   version: 0.2.0
🔖 huggingface_hub version: 0.31.2
🔖 numpy           version: 2.2.5
🔖 scipy           version: 1.15.3
🔖 pandas

# PDF Loader

In [3]:
class PDFLoaderConfig(BaseSettings):
    """
    Configuration for PDFLoader.
    """
    base_dir: Path
    db_path: Path = Path("pdf_index.db")


class PDFRecord(SQLModel, table=True):
    """
    ORM/Pydantic model for a PDF file record in SQLite.
    """
    __table_args__ = {"extend_existing": True}
    
    id: Optional[int] = Field(default=None, primary_key=True)
    path: str = Field(index=True, unique=True, description="File path, relative to base_dir")
    hash: str = Field(description="SHA256 hash of the file contents")


class PDFLoader:
    """
    Scans a directory recursively for PDF files, computes hashes, and keeps
    a local SQLite index in sync.
    """

    def __init__(self, base_dir: Union[str, Path], db_path: Union[str, Path], echo_sql: bool = False):
        self.base_dir = Path(base_dir).expanduser().resolve()
        self.db_path = Path(db_path).expanduser().resolve()
        self.engine = create_engine(f"sqlite:///{self.db_path}", echo=echo_sql)
        SQLModel.metadata.create_all(self.engine)

    def compute_hash(self, pdf_path: Path) -> str:
        """
        Compute SHA256 hash of the given file.
        """
        hasher = hashlib.sha256()
        with pdf_path.open("rb") as f:
            for chunk in iter(lambda: f.read(8192), b""):
                hasher.update(chunk)
        digest = hasher.hexdigest()
        print(f"Computed hash for {pdf_path}: {digest}")
        return digest

    def list_pdf_files(self) -> List[Path]:
        """
        Recursively list all .pdf files under base_dir.
        """
        files = list(self.base_dir.rglob("*.pdf"))
        print(f"Found {len(files)} PDF files under {self.base_dir}")
        return files

    def extract_text(self, pdf_path: Union[str, Path]) -> str:
        """
        Extract and return all the text from a PDF file.
        """
        path = Path(pdf_path)
        reader = PdfReader(str(path))
        text_chunks = []
        for page in reader.pages:
            text_chunks.append(page.extract_text() or "")
        content = "\n".join(text_chunks)
        print(f"Extracted text from {path} ({len(content)} characters)")
        return content

    def _load_db_records(self, session: Session) -> Dict[str, PDFRecord]:
        """
        Load all PDFRecord rows from the database into a path->record dict.
        """
        records = session.exec(select(PDFRecord)).all()
        return {r.path: r for r in records}

    def full_sync(self) -> None:
        """
        Synchronize the on-disk PDFs with the SQLite index. Inserts new files,
        updates changed hashes, and removes records for deleted files.
        """
        # 1. Scan disk
        pdf_paths = self.list_pdf_files()
        # Map relative path -> absolute Path
        rel_to_abs: Dict[str, Path] = {
            str(p.relative_to(self.base_dir)): p for p in pdf_paths
        }
        
        # 2. Compute disk hashes
        disk_hashes: Dict[str, str] = {
            rel: self.compute_hash(abs_p) for rel, abs_p in rel_to_abs.items()
        }

        with Session(self.engine) as session:
            db_map = self._load_db_records(session)

            # 3. Insert or update
            for rel_path, file_hash in disk_hashes.items():
                if rel_path in db_map:
                    record = db_map[rel_path]
                    if record.hash != file_hash:
                        print(f"Updating hash for {rel_path}")
                        record.hash = file_hash
                        session.add(record)
                    # mark as processed
                    del db_map[rel_path]
                else:
                    print(f"Inserting new record for {rel_path}")
                    new_record = PDFRecord(path=rel_path, hash=file_hash)
                    session.add(new_record)

            # 4. Delete records no longer on disk
            for missing_path, record in db_map.items():
                print(f"Deleting record for removed file {missing_path}")
                session.delete(record)

            session.commit()
        print("Full sync complete.")

    def get_record(self, rel_path: str) -> Optional[PDFRecord]:
        """
        Retrieve a single PDFRecord by its relative path.
        """
        with Session(self.engine) as session:
            return session.get(PDFRecord, rel_path)

    def list_records(self) -> List[PDFRecord]:
        """
        Return all indexed PDFRecords.
        """
        with Session(self.engine) as session:
            return session.exec(select(PDFRecord)).all()

In [4]:
pdf_loader_config = PDFLoaderConfig(base_dir='/home/electron/PycharmProjects/fine_tune_llm/RAG', db_path="pdf_index.db")
loader = PDFLoader(pdf_loader_config.base_dir, pdf_loader_config.db_path, echo_sql=False)

## Full sync database with disk

In [5]:
loader.full_sync()

Found 2 PDF files under /home/electron/PycharmProjects/fine_tune_llm/RAG
Computed hash for /home/electron/PycharmProjects/fine_tune_llm/RAG/pdfs/Guide_to_RTA_dec2020.pdf: 98fe6fc26347d4e76741c936de274b085de2c8ba47de56a88c0dd415b6232cb9
Computed hash for /home/electron/PycharmProjects/fine_tune_llm/RAG/pdfs/RENTING-AN-APARTMENT-GUIDE.pdf: c9f75e084d2e19aff270a7d8a3546859a355c39e89a769a275c1ffdf0d15da9e
Inserting new record for pdfs/Guide_to_RTA_dec2020.pdf
Inserting new record for pdfs/RENTING-AN-APARTMENT-GUIDE.pdf
Full sync complete.


## Load PDF Content and metadata

In [6]:
pdf_items = []
for rec in loader.list_records():
    print(f"{rec.id}: {rec.path} [{rec.hash[:8]}...]")
    pdf_path = pdf_loader_config.base_dir / rec.path  # Path object to the PDF
    text = loader.extract_text(pdf_path)  # full text for text/semantic chunkers
    pdf_items.append((rec.id, str(pdf_path), text))
    print(len(text))
    print(text[:100])
    print('#' * 50)

1: pdfs/Guide_to_RTA_dec2020.pdf [98fe6fc2...]
Extracted text from /home/electron/PycharmProjects/fine_tune_llm/RAG/pdfs/Guide_to_RTA_dec2020.pdf (18758 characters)
18758
Brochure: A Guide to the Residential Tenancies Act
Information in this guide
This guide is a summary
##################################################
2: pdfs/RENTING-AN-APARTMENT-GUIDE.pdf [c9f75e08...]
Extracted text from /home/electron/PycharmProjects/fine_tune_llm/RAG/pdfs/RENTING-AN-APARTMENT-GUIDE.pdf (27843 characters)
27843
1 
 
RENTING AN APARTMENT GUIDE  
 
In Ontario, landlord and tenant rights and obligations are gover
##################################################


# Chunker

In [7]:
class Chunk(BaseModel):
    text: str
    rec_id: int
    filename: str
    summary_before: Optional[str] = None
    summary_after: Optional[str] = None
    keywords: Optional[List[str]] = None


class ChunkerConfig(BaseModel):
    chunk_size: int = 1000
    chunk_overlap: int = 200
    semantic_model: str = "gpt-3.5-turbo"  # "gpt-4"
    keywords_model: str = "gpt-3.5-turbo"  #"gpt-4"
    summary_model: str = "gpt-3.5-turbo"  #"gpt-4"
    summary_temperature: float = 0.3
    keywords_temperature: float = 0.0

    db_url: Path = Path("chunks.db")


class ChunkMode(str, Enum):
    text = "text"
    structure = "structure"
    semantic = "semantic"


class ChunkRecord(SQLModel, table=True):
    """
    Persists a single chunk + the exact ChunkerConfig that created it,
    plus a mode flag so we know whether it was text, struct, or semantic chunked.
    """
    __table_args__ = {"extend_existing": True}
    
    id: Optional[int] = Field(default=None, primary_key=True)

    # chunk fields
    rec_id: int
    filename: str
    text: str
    summary_before: Optional[str] = None
    summary_after: Optional[str] = None
    keywords: Optional[List[str]] = Field(sa_column=Column(JSON), default=None)

    # metadata
    mode: ChunkMode = Field(index=True, description="Which chunking strategy created this")

    # config snapshot
    chunk_size: Optional[int] = None
    chunk_overlap: Optional[int] = None
    semantic_model: Optional[str] = None
    keywords_model: Optional[str] = None
    summary_model: Optional[str] = None
    summary_temperature: Optional[float] = None
    keywords_temperature: Optional[float] = None


class Chunker:
    """
    Provides three PDF chunking strategies for a RAG pipeline:
      1. Free-text chunking (fixed size + overlap)
      2. Hierarchical chunking (by document structure)
      3. Semantic chunking (via LLM)
    Also extracts metadata: filename, neighbor summaries, keywords.
    """

    def __init__(self, config: ChunkerConfig = ChunkerConfig(), openai_api_key: Optional[str] = None):
        self.config = config
        self.openai_client = OpenAI(api_key=openai_api_key)

        db_path = Path(self.config.db_url).expanduser().resolve()
        self._engine = create_engine(f"sqlite:///{db_path}", echo=False, future=True)
        SQLModel.metadata.create_all(self._engine)

    def chunk_by_text(self, pdfs: List[Tuple[int, str, str]], chunk_size: Optional[int] = None,
                      chunk_overlap: Optional[int] = None, ) -> List[Chunk]:
        size = chunk_size or self.config.chunk_size
        overlap = chunk_overlap or self.config.chunk_overlap
        try:
            splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)
            chunks: List[Chunk] = []
            for rec_id, filename, text in pdfs:
                pieces = splitter.split_text(text)
                for piece in pieces:
                    chunk = Chunk(text=piece, rec_id=rec_id, filename=filename)
                    chunk.keywords = self._extract_keywords(piece)
                    chunks.append(chunk)
            self._add_neighbor_summaries(chunks)
            return chunks
        except Exception:
            print("Failed free-text chunking")
            return []

    def chunk_by_structure(self, pdfs: List[Tuple[int, str, str]]) -> List[Chunk]:
        try:
            chunks: List[Chunk] = []
            for rec_id, filename, _ in pdfs:
                elements = partition_pdf(filename=filename)
                chunks_per_file = chunk_by_title(elements)
                for cpf in chunks_per_file:
                    chunk = Chunk(text=str(cpf).strip(), rec_id=rec_id, filename=filename)
                    chunk.keywords = self._extract_keywords(chunk.text)
                    chunks.append(chunk)
            self._add_neighbor_summaries(chunks)
            return chunks
        except Exception:
            print("Failed structural chunking")
            return []

    def chunk_semantic(self, pdfs: List[Tuple[int, str, str]]) -> List[Chunk]:
        try:
            chunks: List[Chunk] = []
            for rec_id, filename, text in pdfs:
                prompt = (
                    "Chunk the following text into semantically coherent sections.\n"
                    "Return only a plaintext machine-readable text without any '\n' or special formatting character in your response "
                    "such that it can be directly loaded into a JSON list of strings, each string being one chunk, by simply calling json.loads()\n"
                    "Make sure that your response is only a plaintext format json loadable list of chunks in string format and contains nothing else.\n"
                    f"Text:\n'''{text}'''"
                )
                resp = self.openai_client.chat.completions.create(
                    model=self.config.semantic_model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.0,
                )
                time.sleep(1)
                content = resp.choices[0].message.content
                # Wrap the end of reply in case max token size is reached.
                if not content.endswith('"]'):
                    content += '"]'
                items = json.loads(content)
                for piece in items:
                    chunk = Chunk(text=piece, rec_id=rec_id, filename=filename)
                    chunk.keywords = self._extract_keywords(piece)
                    chunks.append(chunk)
            self._add_neighbor_summaries(chunks)
            return chunks
        except Exception:
            print("Failed semantic chunking")
            print(content)
            return []

    def store_chunks(self, chunks: List[Chunk], mode: ChunkMode) -> None:
        """
        Persist a list of Chunk objects under a given mode (text/struct/semantic),
        stamping each record with the full config snapshot.
        """
        records = []
        for c in chunks:
            if mode == ChunkMode.text:
                kwargs = {
                    'chunk_size': self.config.chunk_size,
                    'chunk_overlap': self.config.chunk_overlap,
                }
            elif mode == ChunkMode.semantic:
                kwargs = {
                    'semantic_model': self.config.semantic_model,
                }
            else:
                kwargs = {}
            records.append(
                ChunkRecord(
                    rec_id=c.rec_id,
                    filename=c.filename,
                    text=c.text,
                    summary_before=c.summary_before,
                    summary_after=c.summary_after,
                    keywords=c.keywords,
                    mode=mode,
                    # snapshot of config
                    keywords_model=self.config.keywords_model,
                    summary_model=self.config.summary_model,
                    summary_temperature=self.config.summary_temperature,
                    keywords_temperature=self.config.keywords_temperature,
                    **kwargs
                )
            )
        with Session(self._engine) as session:
            session.add_all(records)
            session.commit()

    def retrieve_chunks(self) -> Dict[str, List[Chunk]]:
        """
        Load all ChunkRecords from the DB, reconstruct three lists
        in the order (text_chunks, struct_chunks, semantic_chunks).
        """
        text_chunks: List[Chunk] = []
        struct_chunks: List[Chunk] = []
        semantic_chunks: List[Chunk] = []

        stmt = select(ChunkRecord)
        with Session(self._engine) as session:
            for rec in session.exec(stmt):
                c = Chunk(
                    text=rec.text,
                    rec_id=rec.rec_id,
                    filename=rec.filename,
                    summary_before=rec.summary_before,
                    summary_after=rec.summary_after,
                    keywords=rec.keywords,
                )
                if rec.mode == ChunkMode.text:
                    text_chunks.append(c)
                elif rec.mode == ChunkMode.structure:
                    struct_chunks.append(c)
                elif rec.mode == ChunkMode.semantic:
                    semantic_chunks.append(c)

        return {ChunkMode.text: text_chunks, ChunkMode.structure: struct_chunks, ChunkMode.semantic: semantic_chunks}

    def sync_chunks(self, pdfs: List[Tuple[int, str, str]]) -> None:
        """
        Ensure the chunk DB exactly matches the given PDFs:
          - Remove all chunks for any rec_id no longer present.
          - For any new rec_id, run all three chunkers and store their chunks.
        :param pdfs: list of (rec_id, path, hash) tuples representing the source‐of‐truth PDFs
        """
        # 1) Build sets of PDF IDs
        incoming_ids = {rid for rid, _, _ in pdfs}

        # 2) Find which IDs are already in the DB
        with Session(self._engine) as session:
            stmt = select(ChunkRecord.rec_id).distinct()
            existing_ids = {row for row in session.exec(stmt)}

            # 3) Delete orphaned chunks
            to_delete = existing_ids - incoming_ids
            if to_delete:
                print('Deleting items:', to_delete)
                session.exec(
                    delete(ChunkRecord)
                    .where(ChunkRecord.rec_id.in_(to_delete))
                )
            session.commit()

        # 4) Chunk & store for any new PDFs
        to_add = incoming_ids - existing_ids
        if not to_add:
            print('Nothing to add')
            return

        # only keep the new‐PDF tuples
        new_pdfs = [(rid, path, hsh) for rid, path, hsh in pdfs if rid in to_add]

        # run each strategy and persist
        text_chunks = self.chunk_by_text(new_pdfs)
        struct_chunks = self.chunk_by_structure(new_pdfs)
        semantic_chunks = self.chunk_semantic(new_pdfs)

        self.store_chunks(text_chunks, mode=ChunkMode.text)
        self.store_chunks(struct_chunks, mode=ChunkMode.structure)
        self.store_chunks(semantic_chunks, mode=ChunkMode.semantic)

    def _extract_keywords(self, text: str) -> List[str]:
        try:
            prompt = (
                "Extract up to 3 of the most important, specific keywords or phrases from the text below. "
                "If 3 important keywords do not exist, then return fewer keywords. Prioritize the quality and performance of keywords over the number of them. "
                "Return a comma separated list of keywords without saying anything else. Do not use any special character other than comma.\n\n"
                f"Text:\n'''{text}'''"
            )
            resp = self.openai_client.chat.completions.create(
                model=self.config.keywords_model,
                messages=[{"role": "user", "content": prompt}],
                temperature=self.config.keywords_temperature,
            )
            time.sleep(1)
            return list(map(lambda x: x.strip(), resp.choices[0].message.content.split(',')))
        except Exception:
            print("Keyword extraction failed")
            return []

    def _generate_summary(self, text: str) -> str:
        try:
            prompt = (
                "Write a short, concise (one-sentence) summary of the following text. In your response, only provide the summary and say nothing else.\n\n"
                f"Text:\n'''{text}'''"
            )
            resp = self.openai_client.chat.completions.create(
                model=self.config.summary_model,
                messages=[{"role": "user", "content": prompt}],
                temperature=self.config.summary_temperature,
            )
            time.sleep(1)
            return resp.choices[0].message.content.strip()
        except Exception:
            print("Summary generation failed")
            return ""

    def _add_neighbor_summaries(self, chunks: List[Chunk]) -> None:
        # Pre-compute summaries for each chunk
        summaries = [self._generate_summary(c.text) for c in chunks]
        for i, chunk in enumerate(chunks):
            if i > 0:
                chunk.summary_before = summaries[i - 1]
            if i < len(chunks) - 1:
                chunk.summary_after = summaries[i + 1]

In [8]:
api_key = "INSERT-YOUR-OPENAI-TOKEN-HERE"
chunker_config = ChunkerConfig(chunk_size=1000, chunk_overlap=200)
chunker = Chunker(config=chunker_config, openai_api_key=api_key)

## Chunk PDFs and save to database

In [10]:
chunk_methods = [ChunkMode.text, ChunkMode.structure, ChunkMode.semantic]
methods_chunks_mapper = {k: None for k in chunk_methods}
for chunk_name, chunk_func in zip(chunk_methods,
                                  [chunker.chunk_by_text, chunker.chunk_by_structure, chunker.chunk_semantic]):
    print(f'Chunking by {chunk_name}')
    start_time = time.time()
    method_chunks = chunk_func(pdf_items)
    end_time = time.time()
    methods_chunks_mapper[chunk_name] = method_chunks
    chunker.store_chunks(method_chunks, mode=chunk_name)
    print(f'Finished chunking in {end_time - start_time:.2f} seconds and yielded {len(method_chunks)} chunks \n')
    print('Sample chunk:')
    print(method_chunks[1])
    print('\n')
    print('#' * 50)
    print('\n')

Chunking by ChunkMode.text
Finished chunking in 189.72 seconds and yielded 59 chunks 

Sample chunk:
text="homes, and sites in a mobile home park or land lease community.\nMany of the rules about rent do not apply to:\nnon-profit and public housing\nuniversity and college residences\nBut these units are still covered by most of the other rules in the Act about such things as maintenance and the\nreasons for eviction.\nThe Act does not apply if the tenant must share a kitchen or bathroom with the landlord.\nAbout the LTB\nThe Landlord and Tenant Board resolves disputes between tenants and landlords. It is similar to a court.\nEither a landlord or a tenant can apply to the LTB. Their disputes can be worked out through mediation or\nadjudication.\nIn mediation, an LTB mediator helps a landlord and tenant reach an agreement they are both satisfied with.\nIn adjudication, a hearing is held. After the hearing, an LTB member makes a decision based on the evidence\nthat the landlord and tenant

## Load chunks and metadata from database

In [11]:
chunks = chunker.retrieve_chunks()
print(len(chunks), len(chunks[ChunkMode.text]), len(chunks[ChunkMode.structure]), len(chunks[ChunkMode.semantic]))

3 59 133 66


## Full sync chunk database with PDFs

In [12]:
# In case pdf files have changed

chunker.sync_chunks(pdf_items)

Nothing to add


# Embedder

In [13]:
class Embedder:
    """
    Produces embeddings for a list of Chunk objects, using either
    OpenAI's embeddings API or a local sentence-transformers model.
    """

    def __init__(self, backend: Literal["openai", "sentence_transformers"] = "openai",
                 openai_model: str = "text-embedding-ada-002", sentence_model: str = "all-MiniLM-L6-v2",
                 openai_api_key: Optional[str] = None, batch_size: int = 512):
        self.backend = backend
        self.openai_model = openai_model
        self.sentence_model = sentence_model
        self.openai_api_key = openai_api_key or os.getenv("OPENAI_API_KEY")
        self.batch_size = batch_size

        if self.backend == "openai":
            if not self.openai_api_key:
                raise ValueError(
                    "No OpenAI API key provided; set it via argument or OPENAI_API_KEY environment variable"
                )
            self._openai_client = OpenAI(api_key=self.openai_api_key)

        elif self.backend == "sentence_transformers":
            self._encoding_model = SentenceTransformer(self.sentence_model)

        else:
            raise ValueError(f"Unsupported backend: {self.backend}")

    def embed_single_text(self, text: str) -> List[float]:
        """
        Embeds a single string.
        """
        if self.backend == "openai":
            return self._embed_openai([text])[0]
        else:
            return self._embed_sentence_transformers([text])[0]

    def embed(self, chunks: Dict[ChunkMode, Chunk]) -> List[List[float]]:
        """
        Embed a list of Chunk objects in order, returning a parallel list
        of embedding vectors.
        """
        texts = [chunk.text for chunk in chunks]
        if self.backend == "openai":
            return self._embed_openai(texts)
        else:
            return self._embed_sentence_transformers(texts)

    def _embed_openai(self, texts: List[str]) -> List[List[float]]:
        """
        Batch-call OpenAI’s embeddings API and return a list of vectors.
        """
        embeddings: List[List[float]] = []
        for i in range(0, len(texts), self.batch_size):
            batch = texts[i: i + self.batch_size]
            response = self._openai_client.embeddings.create(model=self.openai_model, input=batch)
            embeddings.extend(item["embedding"] for item in response["data"])
        return embeddings

    def _embed_sentence_transformers(self, texts: List[str]) -> List[List[float]]:
        """
        Use a SentenceTransformer to embed texts in a single call.
        """
        embs = self._encoding_model.encode(
            texts,
            batch_size=self.batch_size,
            show_progress_bar=False,
            convert_to_numpy=False,
        )
        return [list(vec) for vec in embs]


In [14]:
embedder = Embedder(backend='sentence_transformers')

## Embed chunks

In [15]:
embeddings = {}
for mode, chunk_pieces in chunks.items():
    embeddings[mode] = embedder.embed(chunk_pieces)

In [16]:
print(len(embeddings[ChunkMode.text]), len(embeddings[ChunkMode.structure]), len(embeddings[ChunkMode.semantic]))

59 133 66


In [17]:
print(len(embeddings[ChunkMode.text][0]), len(embeddings[ChunkMode.structure][0]), len(embeddings[ChunkMode.semantic][0]))

384 384 384


# Vector database (embeddings + metadata)

In [18]:
class VectorStore:
    """
    A local, persistent vector store backed by Qdrant, capable of storing embeddings and
    rich metadata, and fully synchronizing (upsert + delete) based on incoming chunks.
    """

    def __init__(self, *, path: str = "qdrant_db", collection_name: str = "chunks", embedding_dim: int = 384,
                 distance: Distance = Distance.COSINE, embedder: Embedder):
        # Ensure storage directory exists
        os.makedirs(path, exist_ok=True)

        # Connect in local mode (":memory:" or disk) :contentReference[oaicite:1]{index=1}
        self.client = QdrantClient(path=path)
        self.collection_name = collection_name
        self.embedding_dim = embedding_dim

        # Create collection if it does not already exist
        existing = {col.name for col in self.client.get_collections().collections}
        if collection_name not in existing:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=embedding_dim, distance=distance),
            )

        self.embedder = embedder

    def _get_point_id(self, chunk: Chunk) -> str:
        """
        Create a stable, deterministic ID for each chunk by hashing its rec_id,
        filename, and text content.
        """
        digest = hashlib.md5(
            f"{chunk.rec_id}-{chunk.filename}-{chunk.text}".encode("utf-8")
        ).hexdigest()
        return digest

    def sync_embeddings(self, chunks: List[Chunk], embeddings: List[List[float]]) -> None:
        """
        Fully synchronize the vector store:
          1. Delete all points whose rec_id is *not* in the provided chunks.
          2. Upsert all provided chunks (insert new and overwrite existing).
        """
        if len(chunks) != len(embeddings):
            raise ValueError(
                "Number of chunks and embeddings must match."
            )

        # 1. Delete stale points by rec_id
        rec_ids = {chunk.rec_id for chunk in chunks}
        delete_filter = Filter(
            must_not=[
                FieldCondition(
                    key="rec_id",
                    match=MatchAny(any=list(rec_ids)),
                )
            ]
        )
        self.client.delete(
            collection_name=self.collection_name,
            points_selector=delete_filter,
            wait=True,
        )

        # 2. Prepare upsert batch
        points = []
        for chunk, vector in zip(chunks, embeddings):
            point_id = self._get_point_id(chunk)
            payload = {
                "text": chunk.text,
                "rec_id": chunk.rec_id,
                "filename": chunk.filename,
                # These may be None; Qdrant will store them as null
                "summary_before": chunk.summary_before,
                "summary_after": chunk.summary_after,
                "keywords": chunk.keywords,
            }
            points.append(
                PointStruct(id=point_id, vector=vector, payload=payload)
            )

        # Upsert inserts new and replaces existing points by ID
        self.client.upsert(
            collection_name=self.collection_name,
            points=points,
            wait=True,
        )

    def load_all_embeddings(self) -> List:
        """
        Load all embeddings from Qdrant database
        """
        all_embeddings = []
        offset = None

        while True:
            response = self.client.scroll(
                collection_name=self.collection_name,
                scroll_filter=None,  # No filter = retrieve all
                limit=128,  # Adjust batch size as needed
                offset=offset,
                with_payload=True,
                with_vectors=True
            )

            points, offset = response
            all_embeddings.extend(points)

            if offset is None:
                break  # No more points to retrieve
        return all_embeddings

    def _build_metadata_filter(self, query: str) -> Filter:
        """
        Turn raw string into a Qdrant Filter over `keywords` and `filename`.
        Matches any token of length>2 appearing in those fields.
        """
        tokens = {tok.lower() for tok in query.split() if len(tok) > 2}
        conds = []
        for tok in tokens:
            conds.append(FieldCondition(key="keywords", match=MatchAny(any=[tok])))
            conds.append(FieldCondition(key="filename", match=MatchText(text=tok)))
        return Filter(should=conds) if conds else None

    def _mmr(
            self,
            query_vec: np.ndarray,
            candidates: List[np.ndarray],
            scores: List[float],
            k: int,
            diversity: float = 0.5,
    ) -> List[int]:
        """
        Simple MMR selector: pick k indices balancing relevance (scores)
        vs. diversity (cosine sim between candidates).
        """
        # normalize
        cands = np.vstack(candidates)
        cands_norm = cands / np.linalg.norm(cands, axis=1, keepdims=True)
        q_norm = query_vec / np.linalg.norm(query_vec)

        selected = []
        unpicked = set(range(len(scores)))
        # start with best relevance
        first = int(np.argmax(scores))
        selected.append(first)
        unpicked.remove(first)

        while len(selected) < k and unpicked:
            mmr_vals = {}
            for i in unpicked:
                rel = scores[i]
                div = max(
                    np.dot(cands_norm[i], cands_norm[j]) for j in selected
                )
                mmr_vals[i] = diversity * rel - (1 - diversity) * div
            nxt = max(mmr_vals, key=mmr_vals.get)
            selected.append(nxt)
            unpicked.remove(nxt)

        return selected

    def _refine_query(self, query: str) -> str:
        """
        Placeholder for “query refinement” (e.g. LLM paraphrase or keyword extract).
        By default, returns the original query.
        """
        return query

    def retrieve(
            self,
            query: str,
            *,
            top_k: int = 5,
            mode: str = "hybrid",  # "vector" | "metadata" | "hybrid"
            strategy: str = "knn",  # "knn" | "mmr"
            fetch_multiplier: int = 3,  # only used for MMR
            random_rerank: bool = False,
            rerun_refinement: bool = False,
            refinement_threshold: float = 0.5,  # rerun if max_score < this
    ) -> List[Tuple[str, float]]:
        """
        Returns up to top_k (text, score) tuples.
        """
        # step 1: metadata filter if needed
        meta_filter = None
        if mode in ("metadata", "hybrid"):
            meta_filter = self._build_metadata_filter(query)

        # step 2: vector embed if needed
        query_vec = None
        if mode in ("vector", "hybrid"):
            # Not the most efficient way of doing things. Better approach: embedder returns torch vectors on GPU directly
            vec_list = self.embedder.embed_single_text(query)
            query_vec = torch.tensor(vec_list, device='cuda', dtype=torch.float32)

        # step 3: retrieve
        results = []
        if mode == "metadata":
            # purely metadata: scroll + equal confidence
            docs = self.client.scroll(
                collection_name=self.collection_name,
                filter=meta_filter,
                limit=top_k,
                with_payload=True,
            )
            results = [(d.payload["text"], 1.0) for d in docs]

        else:
            # vector or hybrid
            limit = top_k if strategy == "knn" else top_k * fetch_multiplier
            # search_res = self.client.search(
            search_res = self.client.query_points(
                collection_name=self.collection_name,
                # query_vector=query_vec.tolist(),
                query=query_vec.tolist(),
                query_filter=meta_filter,
                limit=limit,
                with_payload=True,
                with_vectors=(strategy == "mmr"),
            )
            # unpack
            texts, scores, vecs = [], [], []
            for scored_point in search_res.points:
                texts.append(scored_point.payload["text"])
                scores.append(scored_point.score)
                if strategy == "mmr":
                    vecs.append(np.array(scored_point.vector, dtype=np.float32))

            if strategy == "knn":
                # just top_k by score
                sliced = list(zip(texts, scores))[:top_k]
                results = sliced

            else:  # MMR
                idxs = self._mmr(query_vec.tolist(), vecs, scores, top_k)
                # idxs = self._mmr_torch(query_vec, vecs, scores, top_k)
                results = [(texts[i], scores[i]) for i in idxs]

        # step 4: optional random rerank
        if random_rerank:
            random.shuffle(results)

        # step 5: optional refinement loop
        if rerun_refinement and results:
            max_score = max(score for _, score in results)
            if max_score < refinement_threshold:
                refined = self._refine_query(query)
                alt = self.retrieve(
                    refined,
                    top_k=top_k,
                    mode=mode,
                    strategy=strategy,
                    fetch_multiplier=fetch_multiplier,
                    random_rerank=random_rerank,
                    rerun_refinement=False,  # prevent loop
                )
                # merge & pick best unique texts
                merged = {txt: scr for txt, scr in (results + alt)}
                # sort & slice
                results = sorted(
                    merged.items(), key=lambda x: x[1], reverse=True
                )[:top_k]

        return results

    def close(self) -> None:
        """
        Close the Qdrant client connection to release any underlying resources.
        """
        try:
            self.client.close()
        except AttributeError:
            # QdrantClient may not expose a close method in some versions
            pass

In [19]:
vector_store = VectorStore(path="qdrant_db", collection_name="chunks", embedder=embedder)

## Full sync vector database with chunks

In [20]:
for mode in chunks.keys():
    print(f"Syncing chunks and embeddings: {mode}")
    vector_store.sync_embeddings(chunks[mode], embeddings[mode])

Syncing chunks and embeddings: ChunkMode.text
Syncing chunks and embeddings: ChunkMode.structure
Syncing chunks and embeddings: ChunkMode.semantic


In [21]:
all_embeddings = vector_store.load_all_embeddings()
len(all_embeddings)

252

In [22]:
len(all_embeddings[1].vector)

384

In [23]:
all_embeddings[1].payload

{'text': "basis\nat least 28 days' notice the end of a weekly rental period\n(This only applies to weekly\ntenancies.)\npays rent on a monthly basis at least 60 days' notice the end of a monthly rental period\nhas a lease for a fixed term at least 60 days' notice no earlier than the last day of the\nlease\nA tenant and landlord can agree to end a tenancy early. The parties can make an oral agreement to end the\ntenancy, but it is best to have a written agreement. A notice of termination does not have to be given by either\nthe landlord or the tenant if there is an agreement to end the tenancy.\nA tenant in a care home can end a tenancy early, by giving at least 30 days' notice in writing to the landlord.\nAssigning a tenancy and subletting\nA tenant may be able to transfer their right to occupy the rental unit to someone else. This is called an\nassignment. In an assignment, a new person takes the place of the tenant, but all the terms of the rental\nagreement stay the same.",
 'rec_id

## Sample queries context retrieval

In [24]:
sample_queries = [
    'I just moved in to my new apartment 3 months ago and now my landlord has increased the rent. Is that legal in Ontario?',
    'My landlord just notified me that my rent will increase effective next month. Do I have to pay him extra coming next month?',
    'My landlord entered my unit earlier today when I was not at work for without previous notice. He mentioned that there was a water leak in the kitchen. Is that legal?',
    'I pay rent to my landlord every week. What is my notice period if I want to vacate the unit?',
]

In [25]:
query_context_mapper = {}
for query in sample_queries:
    print(f'Running query: {query}')
    print(f'Retriving context top 3 context using knn method')
    start_time = time.time()
    retrieved_contexts = vector_store.retrieve(query=query, strategy="knn", top_k=3)
    end_time = time.time()
    query_context_mapper[query] = [context[0] for context in retrieved_contexts]
    print(f'Finished context retrieval in {end_time - start_time:.2f} seconds\n')
    for context in retrieved_contexts:
        print(f'Confidence: {context[1]} \n Context: {context[0]}')
        print('\n')
    print('#' * 50)
    print('\n')

Running query: I just moved in to my new apartment 3 months ago and now my landlord has increased the rent. Is that legal in Ontario?
Retriving context top 3 context using knn method
Finished context retrieval in 0.05 seconds

Confidence: 0.6337752081043402 
 Context: Increase the Rent – There are special rules that limit how often a landlord can increase the rent and by how much. In most cases, a landlord can increase the rent only once a year by the guideline that is set by the Minister of Municipal Affairs and Housing. A landlord must give a tenant at least 90 days notice in writing of any rent increase and this notice must be on the proper form. Non-profit and public housing units, residences at schools, colleges and universities, and certain other accommodation are not covered by all the rent rules.


Confidence: 0.5961476251979027 
 Context: If the landlord gives the tenant a notice to increase the rent, the landlord can also ask the tenant to increase the rent deposit by the sam

## Close vector database connection

In [ ]:
# vector_store.close()

# Generation Pipeline

In [26]:
# Database Models
class Chat(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}

    id: Optional[int] = Field(default=None, primary_key=True)
    created_at: datetime = Field(default_factory=datetime.utcnow)
    use_vllm: bool = Field(default=False)
    vllm_endpoint: Optional[str] = Field(default=None)


class ChatRole(str, Enum):
    system = "system"
    user = "user"
    assistant = "assistant"


class Message(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}

    id: Optional[int] = Field(default=None, primary_key=True)
    chat_id: int = Field(foreign_key="chat.id")
    role: ChatRole = Field(description="Which chat role")
    content: str
    timestamp: datetime = Field(default_factory=datetime.utcnow)


# Generation Pipeline
class Generation:
    def __init__(
            self,
            db_url: str = "sqlite:///./chats.db",
            openai_model: str = "gpt-3.5-turbo",  # "gpt-4"
            max_tokens: int = 2000,
            summarization_model: str = "sshleifer/distilbart-cnn-12-6",  # "facebook/bart-large-cnn"
    ):
        # Initialize database
        self.engine = create_engine(db_url, echo=False)
        SQLModel.metadata.create_all(self.engine)

        # LLM settings
        self.openai_model = openai_model
        self.max_tokens = max_tokens

        # Tokenizer for token counting
        self.tokenizer = tiktoken.encoding_for_model(openai_model)

        # Summarization pipeline
        self.summarizer = hf_pipeline("summarization", model=summarization_model)

    # Database Helpers
    def create_chat(self, use_vllm: bool = False, vllm_endpoint: Optional[str] = None) -> int:
        """
        Create a new chat entry in the database.
        Returns the new chat_id.
        """
        chat = Chat(use_vllm=use_vllm, vllm_endpoint=vllm_endpoint)
        with Session(self.engine) as session:
            session.add(chat)
            session.commit()
            session.refresh(chat)
        return chat.id

    def list_chats(self) -> List[Dict[str, Any]]:
        """
        Fetch all chats with their IDs and settings.
        """
        with Session(self.engine) as session:
            chats = session.exec(select(Chat)).all()
        return [
            {"id": c.id, 'created_at': c.created_at.strftime('%Y-%m-%d %H:%M:%S'), "use_vllm": c.use_vllm, "vllm_endpoint": c.vllm_endpoint}
            for c in chats
        ]

    def _get_chat_history(self, chat_id: int) -> List[Dict[str, str]]:
        """
        Retrieve ordered chat history for a given chat_id.
        """
        with Session(self.engine) as session:
            msgs = session.exec(
                select(Message).where(Message.chat_id == chat_id).order_by(Message.id)
            ).all()
        return [{"role": m.role, "content": m.content} for m in msgs]

    def _append_message(self, chat_id: int, role: str, content: str) -> None:
        """
        Append a message to chat history in DB.
        """
        msg = Message(chat_id=chat_id, role=role, content=content)
        with Session(self.engine) as session:
            session.add(msg)
            session.commit()

    # Context Window Management
    def _count_tokens(self, text: str) -> int:
        """
        Count tokens in a text string.
        """
        return len(self.tokenizer.encode(text))

    def _clip_or_summarize(
            self,
            history: List[Dict[str, str]],
            mode: Literal["clip", "summarize"] = "clip",
    ) -> List[Dict[str, str]]:
        """
        Ensure history fits within max_tokens by clipping or summarization.
        """
        # Compute total tokens
        total = sum(self._count_tokens(m["content"]) for m in history)
        if total <= self.max_tokens:
            return history

        if mode == "clip":
            # Drop oldest messages until within limit
            clipped = history.copy()
            while sum(self._count_tokens(m["content"]) for m in clipped) > self.max_tokens:
                clipped.pop(0)
            print(f'Clipping Context window from {total} tokens to {sum(self._count_tokens(m["content"]) for m in clipped)} tokens.')
            return clipped

        # Summarization mode: summarize oldest half
        half_limit = self.max_tokens // 2
        oldest, rest = [], []
        token_acc = 0
        for msg in history:
            size = self._count_tokens(msg["content"])
            if token_acc + size <= half_limit:
                oldest.append(msg)
                token_acc += size
            else:
                rest.append(msg)
        # Create a concatenated text to summarize
        text_to_sum = "\n".join(f"{m['role']}: {m['content']}" for m in oldest)
        summary = self.summarizer(
            text_to_sum, max_length=150, min_length=30, do_sample=False
        )[0]["summary_text"]
        # Prepend summary as system message
        summarized = [{"role": "system", "content": f"Summary of earlier conversation: {summary}"}] + rest
        print(f'Summarized Context window from {total} tokens to {sum(self._count_tokens(m["content"]) for m in summarized)} tokens.')
        return summarized

    # LLM Invocation
    def _call_llm(
            self, messages: List[Dict[str, str]], chat_id: int
    ) -> str:
        """
        Call the configured LLM (OpenAI or local vLLM) with chat messages.
        """
        # Fetch chat settings
        with Session(self.engine) as session:
            chat = session.get(Chat, chat_id)

        # Prepare LangChain messages
        lc_msgs: List[BaseMessage] = []
        for m in messages:
            if m["role"] == "system":
                lc_msgs.append(SystemMessage(content=m["content"]))
            elif m["role"] == "user":
                lc_msgs.append(HumanMessage(content=m["content"]))
            else:
                lc_msgs.append(AIMessage(content=m["content"]))

        if chat.use_vllm and chat.vllm_endpoint:
            # Local vLLM endpoint
            payload = {"messages": messages}
            resp = requests.post(chat.vllm_endpoint, json=payload)
            resp.raise_for_status()
            return resp.json()["choices"][0]["message"]["content"]

        # Default to OpenAI
        client = ChatOpenAI(model_name=self.openai_model, openai_api_key=api_key)
        response = client(lc_msgs)
        return response.content

    # Public Generate API
    def generate(
            self,
            query: str,
            context: Optional[str] = None,
            chat_id: Optional[int] = None,
            mode: Literal["clip", "summarize"] = "clip",
            use_vllm: bool = False,
            vllm_endpoint: Optional[str] = None,
    ) -> Dict[str, Any]:
        """
        Generate a response for a query with optional context.
        If chat_id is None, a new chat is created.
        Returns dict with keys: chat_id, response, history.
        """
        # Initialize or resume chat
        if chat_id is None:
            chat_id = self.create_chat(use_vllm=use_vllm, vllm_endpoint=vllm_endpoint)

        # Append system context
        if context:
            self._append_message(chat_id, "system", context)
        # Append user query
        self._append_message(chat_id, "user", query)

        # Retrieve and manage history
        history = self._get_chat_history(chat_id)
        pruned = self._clip_or_summarize(history, mode)

        # Invoke LLM
        resp_text = self._call_llm(pruned, chat_id)
        # Append assistant response
        self._append_message(chat_id, "assistant", resp_text)

        return {"chat_id": chat_id, "response": resp_text, "history": pruned}

    # Utility to fetch chat history externally
    def fetch_history(self, chat_id: int) -> List[Dict[str, str]]:
        """
        Public method to retrieve full chat history.
        """
        return self._get_chat_history(chat_id)

    def generate_temporary(self, query: str) -> str:
        client = ChatOpenAI(model_name=self.openai_model, openai_api_key=api_key)
        response = client([HumanMessage(content=query)])
        return response.content

In [27]:
# Setting max token to 100 to test context summarization/clipping
generation = Generation(max_tokens=1000)

Device set to use cuda:0


## RAG vs. No-Context generation comparison

In [28]:
chat_responses = {}
for query, contexts in query_context_mapper.items():
    print(f'Running Query: {query} \n')
    start_time = time.time()
    chat_info = generation.generate(query=query, context='\n'.join(contexts))
    end_time = time.time()
    print(f'Finished generation in {end_time - start_time:.2f} seconds\n')
    print(f'Response for chat_id={chat_info['chat_id']} WITH context:')
    chat_responses[query] = chat_info
    print(chat_info['response'])
    print('\n')
    no_context_response = generation.generate_temporary(query=query)
    print(f'Response for chat_id={chat_info['chat_id']} WITHOUT context:')
    print(no_context_response)
    print('\n')
    print('#' * 50)
    print('\n')

Running Query: I just moved in to my new apartment 3 months ago and now my landlord has increased the rent. Is that legal in Ontario? 



/tmp/ipykernel_30454/2289380599.py:171: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  client = ChatOpenAI(model_name=self.openai_model, openai_api_key=api_key)
/tmp/ipykernel_30454/2289380599.py:172: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = client(lc_msgs)


Finished generation in 1.89 seconds

Response for chat_id=1 WITH context:
In Ontario, landlords can only increase the rent once a year by the guideline set by the Minister of Municipal Affairs and Housing. They must give tenants at least 90 days notice in writing of any rent increase. If your landlord has increased the rent within only 3 months of you moving in, it may not be legal unless there are specific circumstances that allow for it. You may want to check the rental laws in Ontario or consult with a legal professional to understand your rights in this situation.


Response for chat_id=1 WITHOUT context:
In Ontario, landlords are allowed to increase the rent once every 12 months for existing tenants, as long as they provide proper notice and follow the rules set out in the Residential Tenancies Act. The landlord must provide at least 90 days' written notice before the rent increase takes effect. Additionally, the rent increase must be within the guidelines set by the government, w

## Notice the more accurate response for RAG vs. non-RAG

**Question: I pay rent to my landlord every week. What is my notice period if I want to vacate the unit?**

**RAG Response**:

If you pay rent on a weekly basis, you must give your landlord **at least 28 days' notice** to vacate the unit. The termination date must be the end of a weekly rental period. This notice period applies specifically to weekly tenancies.


**non-RAG Response**:

Your notice period will depend on the terms of your rental agreement with your landlord. In most cases, landlords require tenants to give 30 days' notice before vacating the unit. However, some landlords may require longer notice periods, **such as 60 or 90 days**. It is important to review your rental agreement or speak with your landlord to determine the specific notice period required in your situation.


## Load chat history

In [29]:
chat_history = []
chats = generation.list_chats()
for c in chats:
    history = generation.fetch_history(chat_id=c["id"])
    chat_history.append({
        "chat_id": c["id"],
        "created_at": c["created_at"],
        "history": history
    })
chat_history[2] # Entry to unit

{'chat_id': 3,
 'created_at': '2025-05-28 02:21:55',
 'history': [{'role': <ChatRole.system: 'system'>,
   'content': "hot or cold water\nMore information about maintenance and repairs\nFor more information read the brochure called Maintenance and Repairs.\nAbout entering the rental unit\nEntry without written notice\nA landlord can enter a tenant's rental unit without written notice if:\nthere is an emergency such as a fire\nthe tenant agrees to let the landlord in\na care home tenant has agreed in writing that the landlord can come in to check on their condition at\nregular intervals\nA landlord can enter a rental unit without written notice, between 8 a.m. and 8 p.m. if:\nthe rental agreement requires the landlord to clean the unit - unless the agreement allows different\nhours for cleaning,\nthe landlord or tenant has given a notice of termination, or they have an agreement to end the tenancy,\nand the landlord wants to show the unit to a potential new tenant (in this case, althoug

## Resume a chat

In [30]:
chat_resumption_info = generation.generate(query='In what situations can the landlord enter my home without notice? Elaborate.',
                                           chat_id=chat_history[2]['chat_id'])
chat_resumption_info['response']

"A landlord can enter a tenant's rental unit without written notice in the following situations:\n\n1. Emergency: If there is an emergency such as a fire or a water leak that requires immediate attention to prevent damage to the property or ensure the safety of occupants, the landlord can enter without prior notice.\n\n2. Tenant's Agreement: If the tenant agrees to let the landlord in without written notice, the landlord can enter the rental unit.\n\n3. Care Home Tenant Agreement: In the case of a care home tenant, if they have agreed in writing that the landlord can come in to check on their condition at regular intervals, the landlord can enter without notice.\n\nAdditionally, a landlord can enter a rental unit without written notice between 8 a.m. and 8 p.m. in the following situations:\n\n1. Required Cleaning: If the rental agreement requires the landlord to clean the unit, the landlord can enter without notice, unless the agreement specifies different hours for cleaning.\n\n2. Not

In [31]:
chat_resumption_info['history']

[{'role': <ChatRole.system: 'system'>,
  'content': "hot or cold water\nMore information about maintenance and repairs\nFor more information read the brochure called Maintenance and Repairs.\nAbout entering the rental unit\nEntry without written notice\nA landlord can enter a tenant's rental unit without written notice if:\nthere is an emergency such as a fire\nthe tenant agrees to let the landlord in\na care home tenant has agreed in writing that the landlord can come in to check on their condition at\nregular intervals\nA landlord can enter a rental unit without written notice, between 8 a.m. and 8 p.m. if:\nthe rental agreement requires the landlord to clean the unit - unless the agreement allows different\nhours for cleaning,\nthe landlord or tenant has given a notice of termination, or they have an agreement to end the tenancy,\nand the landlord wants to show the unit to a potential new tenant (in this case, although notice is not\nrequired, the landlord must try to tell the tenan

In [32]:
chat_resumption_info = generation.generate(query='And what situations require a 24 hour notice for the landlord to enter my home? Elaborate.',
                                           chat_id=chat_history[2]['chat_id'])
chat_resumption_info['response']

"In most cases, a landlord is required to provide a tenant with a 24-hour written notice before entering the rental unit for non-emergency reasons. Some common situations that require a 24-hour notice for the landlord to enter the tenant's home include:\n\n1. Repairs and Maintenance: If the landlord needs to enter the rental unit to perform repairs, maintenance, or inspections that are not urgent or related to an emergency, they must typically provide the tenant with a 24-hour written notice.\n\n2. Inspections: Routine inspections of the rental unit, such as annual inspections for safety or maintenance purposes, usually require a 24-hour notice from the landlord to the tenant.\n\n3. Showing the Unit to Prospective Tenants: If the landlord wants to show the rental unit to potential new tenants, they are generally required to give the current tenant a 24-hour written notice before entering the premises.\n\n4. Pest Control Services: If pest control services need to be conducted in the ren

In [33]:
chat_resumption_info['history']

[{'role': <ChatRole.system: 'system'>,
  'content': "hot or cold water\nMore information about maintenance and repairs\nFor more information read the brochure called Maintenance and Repairs.\nAbout entering the rental unit\nEntry without written notice\nA landlord can enter a tenant's rental unit without written notice if:\nthere is an emergency such as a fire\nthe tenant agrees to let the landlord in\na care home tenant has agreed in writing that the landlord can come in to check on their condition at\nregular intervals\nA landlord can enter a rental unit without written notice, between 8 a.m. and 8 p.m. if:\nthe rental agreement requires the landlord to clean the unit - unless the agreement allows different\nhours for cleaning,\nthe landlord or tenant has given a notice of termination, or they have an agreement to end the tenancy,\nand the landlord wants to show the unit to a potential new tenant (in this case, although notice is not\nrequired, the landlord must try to tell the tenan

## Test context window shortening (Clipping default)

In [34]:
chat_resumption_info = generation.generate(query='Does the notice to enter have to have a reason for entry? Elaborate.',
                                           chat_id=chat_history[2]['chat_id'])
chat_resumption_info['response']

Clipping Context window from 1128 tokens to 832 tokens.


"In most jurisdictions, the notice to enter the rental unit provided by the landlord does not necessarily have to specify a reason for entry. Landlord-tenant laws typically require landlords to provide a certain amount of advance notice before entering the rental unit, but they may not always mandate that the notice include a specific reason for the entry.\n\nHowever, some rental agreements or local laws may require landlords to provide a reason for entry in the notice. Even if not required, it is generally considered good practice for landlords to communicate the purpose of their entry in the notice to provide transparency and maintain a positive landlord-tenant relationship.\n\nIn situations where a reason for entry is not provided in the notice, tenants may feel more comfortable if the landlord communicates the purpose of the visit when they arrive at the rental unit. This can help to alleviate any concerns or uncertainties the tenant may have about the landlord's presence in their 

In [35]:
chat_resumption_info['history']

[{'role': <ChatRole.user: 'user'>,
  'content': 'My landlord entered my unit earlier today when I was not at work for without previous notice. He mentioned that there was a water leak in the kitchen. Is that legal?'},
 {'role': <ChatRole.assistant: 'assistant'>,
  'content': "If there was a water leak in your unit and your landlord entered without previous notice to address the emergency, it may be considered legal. Landlords are allowed to enter a rental unit without written notice in case of emergencies, such as a fire or a water leak that could cause damage to the property. However, it is important for landlords to provide notice whenever possible and to respect the tenant's privacy rights. If you have concerns about your landlord's actions, you may want to review your rental agreement and the applicable landlord-tenant laws in your area, or consider discussing the situation with a legal professional."},
 {'role': <ChatRole.user: 'user'>,
  'content': 'In what situations can the lan

## Test context window shortening (Summarization)

In [36]:
chat_resumption_info = generation.generate(query='What hours of the day can the landlord enter my home even with a notice? Elaborate.',
                                           chat_id=chat_history[2]['chat_id'],
                                           mode="summarize")
chat_resumption_info['response']

Summarized Context window from 1387 tokens to 988 tokens.


"In most jurisdictions, landlords are typically allowed to enter a rental unit between certain specified hours of the day, even with a valid notice provided to the tenant. The specific hours during which a landlord can enter a tenant's home may vary depending on local landlord-tenant laws or the terms outlined in the rental agreement. However, the following general guidelines are commonly observed:\n\n1. **Standard Hours**: Landlords are usually permitted to enter a rental unit between reasonable hours, typically defined as between 8 a.m. and 8 p.m. This timeframe is considered reasonable and respects the tenant's right to privacy and peaceful enjoyment of their home.\n\n2. **24-Hour Notice**: Landlords are typically required to provide tenants with advance notice before entering the rental unit, usually at least 24 hours in advance. The notice should specify the date and time of entry, allowing tenants to prepare for the landlord's visit.\n\n3. **Emergency Exceptions**: In cases of em

In [37]:
chat_resumption_info['history']

[{'role': 'system',
  'content': "Summary of earlier conversation:  Landlords are allowed to enter a rental unit without written notice in case of emergencies, such as a fire or a water leak that could cause damage to the property . Landlords can enter the rental unit between 8 a.m. and 8 p.m., and only if they have given the tenant 24 hours' written notice . A landlord can only enter the unit for the reasons allowed by the Act ."},
 {'role': <ChatRole.assistant: 'assistant'>,
  'content': "A landlord can enter a tenant's rental unit without written notice in the following situations:\n\n1. Emergency: If there is an emergency such as a fire or a water leak that requires immediate attention to prevent damage to the property or ensure the safety of occupants, the landlord can enter without prior notice.\n\n2. Tenant's Agreement: If the tenant agrees to let the landlord in without written notice, the landlord can enter the rental unit.\n\n3. Care Home Tenant Agreement: In the case of a ca

## Test context window shortening (Clipping explicit)

In [38]:
chat_resumption_info = generation.generate(query='What days of the week can the landlord enter my home even with a notice? Elaborate.',
                                           chat_id=chat_history[2]['chat_id'],
                                           mode="clip")
chat_resumption_info['response']

Clipping Context window from 1784 tokens to 673 tokens.


"The days of the week on which a landlord can enter a rental unit with notice can vary depending on local landlord-tenant laws and the terms outlined in the rental agreement. However, some general guidelines are commonly followed:\n\n1. **Business Days**: Landlords typically schedule entry into a rental unit during regular business days, which are considered to be Monday through Friday. This is to ensure that tenants are not inconvenienced during weekends or holidays when they may be spending time at home or entertaining guests.\n\n2. **Excluding Holidays**: Landlords may avoid entering a rental unit on public holidays or days when the rental office is closed. This is out of respect for both the tenant's personal time and the general observance of holidays.\n\n3. **Advance Notice**: Landlords are usually required to provide tenants with advance notice before entering the rental unit, typically at least 24 hours in advance. The notice should specify the date, time, and reason for entry,

In [39]:
chat_resumption_info['history']

[{'role': <ChatRole.user: 'user'>,
  'content': 'Does the notice to enter have to have a reason for entry? Elaborate.'},
 {'role': <ChatRole.assistant: 'assistant'>,
  'content': "In most jurisdictions, the notice to enter the rental unit provided by the landlord does not necessarily have to specify a reason for entry. Landlord-tenant laws typically require landlords to provide a certain amount of advance notice before entering the rental unit, but they may not always mandate that the notice include a specific reason for the entry.\n\nHowever, some rental agreements or local laws may require landlords to provide a reason for entry in the notice. Even if not required, it is generally considered good practice for landlords to communicate the purpose of their entry in the notice to provide transparency and maintain a positive landlord-tenant relationship.\n\nIn situations where a reason for entry is not provided in the notice, tenants may feel more comfortable if the landlord communicates